In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import seaborn as sns
#from statsmodels.graphics.mosaicplot import mosaic
import io
import requests

%matplotlib inline


import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import tree

### Lectura de datos del CSV y seteo del tipo para cada columna

In [65]:
url="https://raw.githubusercontent.com/joacols/TP_orga_de_datos/main/Entrenamieto_ECI_2020.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),

#df = pd.read_csv('Entrenamieto_ECI_2020.csv',
dtype={
'ID': np.int64,
'Pricing, Delivery_Terms_Quote_Approval': np.int64,
'Pricing, Delivery_Terms_Approved': np.int64,
'Bureaucratic_Code_0_Approval': np.int64,
'Bureaucratic_Code_0_Approved': np.int64,
'Submitted_for_Approval': np.int64,
'Opportunity_ID': np.int64,
#'Sales_Contract_No': np.int64,
'Delivery_Year': np.int64,
'Total_Power': np.int64,
#'Price':np.float64,
'ASP':np.float64,
'ASP_(converted)':np.float64,
'Total_Amount':np.float64},
parse_dates=[
'Account_Created_Date',
'Planned_Delivery_Start_Date',
'Planned_Delivery_End_Date',
'Opportunity_Created_Date',
'Last_Activity',
'Quote_Expiry_Date',
'Actual_Delivery_Date',
'Last_Modified_Date'
])

df.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,2015-06-16,...,Q2,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,2015-06-16,...,Q1,2016,NaT,0,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,2015-04-21,...,Q1,2016,NaT,0,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,2013-07-27,...,Q1,2018,NaT,14,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,2013-07-27,...,Q1,2018,NaT,25,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None


## Realizamos el preprocesamiento sobre los datos 

In [66]:
print(df.dtypes)


ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Submitted_for_Approval                         int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  

In [67]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
df2 = df
df2['Region'] = lbl.fit_transform(df2['Region'].astype(str))
df2['Territory'] = lbl.fit_transform(df2['Territory'].astype(str))
df2['Bureaucratic_Code'] = lbl.fit_transform(df2['Bureaucratic_Code'].astype(str))
df2['Source '] = lbl.fit_transform(df2['Source '].astype(str))
df2['Billing_Country'] = lbl.fit_transform(df2['Billing_Country'].astype(str))
df2['Account_Name'] = lbl.fit_transform(df2['Account_Name'].astype(str))
df2['Opportunity_Name'] = lbl.fit_transform(df2['Opportunity_Name'].astype(str))
df2['Account_Owner'] = lbl.fit_transform(df2['Account_Owner'].astype(str))
df2['Opportunity_Owner'] = lbl.fit_transform(df2['Opportunity_Owner'].astype(str))
df2['Account_Type'] = lbl.fit_transform(df2['Account_Type'].astype(str))
df2['Opportunity_Type'] = lbl.fit_transform(df2['Opportunity_Type'].astype(str))
df2['Quote_Type'] = lbl.fit_transform(df2['Quote_Type'].astype(str))
df2['Delivery_Terms'] = lbl.fit_transform(df2['Delivery_Terms'].astype(str))
df2['Brand'] = lbl.fit_transform(df2['Brand'].astype(str))
df2['Product_Type'] = lbl.fit_transform(df2['Product_Type'].astype(str))
df2['Size'] = lbl.fit_transform(df2['Size'].astype(str))
df2['Product_Category_B'] = lbl.fit_transform(df2['Product_Category_B'].astype(str))
df2['Price'] = lbl.fit_transform(df2['Price'].astype(str))
df2['Currency'] = lbl.fit_transform(df2['Currency'].astype(str))
df2['Last_Modified_By'] = lbl.fit_transform(df2['Last_Modified_By'].astype(str))
df2['Product_Family'] = lbl.fit_transform(df2['Product_Family'].astype(str))
df2['Product_Name'] = lbl.fit_transform(df2['Product_Name'].astype(str))
df2['ASP_Currency'] = lbl.fit_transform(df2['ASP_Currency'].astype(str))
df2['ASP_(converted)_Currency'] = lbl.fit_transform(df2['ASP_(converted)_Currency'].astype(str))
df2['Month'] = lbl.fit_transform(df2['Month'].astype(str))
df2['Delivery_Quarter'] = lbl.fit_transform(df2['Delivery_Quarter'].astype(str))
df2['Total_Amount_Currency'] = lbl.fit_transform(df2['Total_Amount_Currency'].astype(str))
df2['Total_Taxable_Amount_Currency'] = lbl.fit_transform(df2['Total_Taxable_Amount_Currency'].astype(str))
df2['Prod_Category_A'] = lbl.fit_transform(df2['Prod_Category_A'].astype(str))

df2['Account_Created_Date'] = lbl.fit_transform(df2['Account_Created_Date'].astype(str))
df2['Opportunity_Created_Date'] = lbl.fit_transform(df2['Opportunity_Created_Date'].astype(str))
df2['Last_Activity'] = lbl.fit_transform(df2['Last_Activity'].astype(str))
df2['Quote_Expiry_Date'] = lbl.fit_transform(df2['Quote_Expiry_Date'].astype(str))
df2['Last_Modified_Date'] = lbl.fit_transform(df2['Last_Modified_Date'].astype(str))
df2['Planned_Delivery_Start_Date'] = lbl.fit_transform(df2['Planned_Delivery_Start_Date'].astype(str))
df2['Planned_Delivery_End_Date'] = lbl.fit_transform(df2['Planned_Delivery_End_Date'].astype(str))
df2['Actual_Delivery_Date'] = lbl.fit_transform(df2['Actual_Delivery_Date'].astype(str))

In [68]:
print(df2.dtypes)


ID                                      int64
Region                                  int64
Territory                               int64
Pricing, Delivery_Terms_Quote_Appr      int64
Pricing, Delivery_Terms_Approved        int64
Bureaucratic_Code_0_Approval            int64
Bureaucratic_Code_0_Approved            int64
Submitted_for_Approval                  int64
Bureaucratic_Code                       int64
Account_Created_Date                    int64
Source                                  int64
Billing_Country                         int64
Account_Name                            int64
Opportunity_Name                        int64
Opportunity_ID                          int64
Sales_Contract_No                      object
Account_Owner                           int64
Opportunity_Owner                       int64
Account_Type                            int64
Opportunity_Type                        int64
Quote_Type                              int64
Delivery_Terms                    

## Separamos la variable a predecir

In [69]:
X = df2.drop(['Stage','Sales_Contract_No'], axis=1)

In [70]:
y = df2['Stage']

In [71]:
X.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,27761,2,47,1,1,1,1,0,4,146,...,7,1,2016,0,10,1,5272800.0,1,5272800.0,0
1,27760,2,47,0,0,0,0,0,4,146,...,0,0,2016,0,0,1,48230.0,1,48230.0,0
2,27446,1,44,0,0,0,0,0,4,130,...,0,0,2016,0,0,4,83865.6,4,83865.6,0
3,16808,1,44,1,0,1,0,0,5,0,...,28,0,2018,0,14,4,7421881.5,4,7421881.5,0
4,16805,1,44,1,0,1,0,0,5,0,...,28,0,2018,0,25,4,13357192.5,4,13357192.5,0


In [72]:
y.head()

0    Closed Lost
1     Closed Won
2     Closed Won
3    Closed Lost
4    Closed Lost
Name: Stage, dtype: object

## Creamos set de entrenamiento y test

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

## Entrenando y prediciendo con hiper-parámetros por defecto

In [14]:
#rf_model = RandomForestRegressor(random_state=1)
#rf_model.fit(X_train, y_train)
#preds = rf_model.predict(X_test)
#rmse = np.sqrt(mean_squared_error(y_test, preds))
#print("RMSE: %f" % (rmse))
#accuracy_score(y_test, preds)

In [74]:
clfXGB = XGBClassifier()
clfXGB.fit(X_train, y_train)

/home/marcelo/my_project_dir/my_project_env/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:20:47] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [75]:
y_pred = clfXGB.predict(X_test)

In [76]:
accuracy_score(y_test, y_pred)

0.9247787610619469

## Multi-layer Perceptron

In [77]:
df3 = df2.dropna()

In [78]:
X = df3.drop(['Stage','Sales_Contract_No'], axis=1)

In [79]:
y = df3['Stage']

In [80]:

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [81]:
from sklearn.neural_network import MLPClassifier

In [82]:
clfMLP = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)


In [83]:
clfMLP.fit(X_train, y_train)

/home/marcelo/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [84]:
y_pred =clfMLP.predict(X_test)


In [85]:
accuracy_score(y_test, y_pred)

0.037997807818779684

## DecisionTreeClassifier 

Realizando predicciones con un DecisionTreeClassifier

In [86]:
from sklearn import tree


In [87]:
clfDT = tree.DecisionTreeClassifier()

In [88]:
clfDT.fit(X_train, y_train)

DecisionTreeClassifier()

In [89]:
y_pred =clfDT.predict(X_test)


In [90]:
accuracy_score(y_test, y_pred)

0.845451223967848

## Ensemble Voting Classifier

In [91]:
from sklearn.ensemble import VotingClassifier

In [92]:
eclf = VotingClassifier(estimators=[('dt', clfDT), ('mlp', clfMLP)], voting='hard')

In [93]:
eclf = eclf.fit(X_train, y_train)

/home/marcelo/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [94]:
y_pred =eclf.predict(X_test)


In [95]:
accuracy_score(y_test, y_pred)

0.7047862623310194

Combinando el algoritmo de DecisionTree con el de MultiLayerPerception la efectividad fue inferior al del algoritmo de DecisionTree por separado.